# EDA v1 - RentSense

Анализ данных о предложениях аренды недвижимости в Москве.

Цель: понять структуру данных, выявить проблемы и подготовиться к обучению моделей.


In [1]:
# Импорты библиотек
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from dotenv import dotenv_values
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
try:
    plt.style.use('seaborn-v0_8')
except:
    try:
        plt.style.use('seaborn')
    except:
        plt.style.use('default')

sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Библиотеки загружены")
print(f"NumPy: {np.__version__}, Pandas: {pd.__version__}, Matplotlib: {matplotlib.__version__}")


Библиотеки загружены
NumPy: 1.26.4, Pandas: 2.3.2, Matplotlib: 3.9.2


## 1. Подключение к БД и загрузка данных


In [2]:
# Подключение к БД (удаленный сервер 89.110.92.128)
from pathlib import Path
import sys
import subprocess

# Проверка и установка pymysql если нужно
try:
    import pymysql
except ImportError:
    print("Устанавливаю pymysql...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pymysql", "--quiet"])
    import pymysql

env_path = Path('../..') / '.env'
env = dotenv_values(env_path)

DBTYPE = env.get('DB_TYPE') or 'mysql+pymysql'
LOGIN = env.get('DB_LOGIN') or 'root'
PASS = env.get('DB_PASS') or 'rootpassword'
IP = env.get('DB_IP') or '89.110.92.128'
PORT = env.get('DB_PORT') or '3306'
DBNAME = env.get('DB_NAME') or 'rentsense'

# Показываем, какие параметры используются (без пароля)
print(f"Параметры подключения:")
print(f"  IP: {IP}")
print(f"  PORT: {PORT}")
print(f"  USER: {LOGIN}")
print(f"  DB: {DBNAME}")

DATABASE_URL = f'{DBTYPE}://{LOGIN}:{PASS}@{IP}:{PORT}/{DBNAME}?charset=utf8mb4'

print(f"Подключение к БД: {DBNAME}@{IP}:{PORT}")
print(f"Параметры из .env: IP={IP}, PORT={PORT}")

try:
    engine = create_engine(DATABASE_URL, pool_pre_ping=True, connect_args={"connect_timeout": 10})
    # Проверка подключения
    with engine.connect() as conn:
        print("Подключение успешно")
except Exception as e:
    print(f"Ошибка подключения: {e}")
    print("\nПроверьте:")
    print("1. Файл .env с параметрами DB_IP, DB_PORT, DB_LOGIN, DB_PASS")
    print("2. Доступность сервера БД (89.110.92.128:3306)")
    print("3. Правильность учетных данных")
    print("4. Firewall/сеть позволяет подключение к удаленному серверу")
    raise


Параметры подключения:
  IP: 89.110.92.128
  PORT: 3306
  USER: rentsense
  DB: rentsense
Подключение к БД: rentsense@89.110.92.128:3306
Параметры из .env: IP=89.110.92.128, PORT=3306


Подключение успешно


In [3]:
# Загрузка данных
# Проверка, что engine определен (ячейка 3 должна выполниться успешно)
if 'engine' not in locals() and 'engine' not in globals():
    raise NameError("Переменная 'engine' не определена. Сначала запустите ячейку 3 (подключение к БД)")

query = """
SELECT 
    o.cian_id,
    o.price,
    o.price_changes,
    o.category,
    o.views_count,
    o.photos_count,
    o.floor_number,
    o.floors_count,
    o.publication_at,
    o.created_at as offer_created_at,
    o.updated_at as offer_updated_at,
    
    a.county,
    a.district,
    a.street,
    a.house,
    a.metro,
    a.travel_type,
    a.travel_time,
    a.coordinates,
    
    ri.repair_type,
    ri.total_area,
    ri.living_area,
    ri.kitchen_area,
    ri.ceiling_height,
    ri.balconies,
    ri.loggias,
    ri.rooms_count,
    ri.separated_wc,
    ri.combined_wc,
    ri.windows_view,
    
    ro.build_year,
    ro.entrances,
    ro.material_type,
    ro.parking_type,
    ro.garbage_chute,
    ro.lifts_count,
    ro.passenger_lifts,
    ro.cargo_lifts,
    
    rd.realty_type,
    rd.project_type,
    rd.heat_type,
    rd.gas_type,
    rd.is_apartment,
    rd.is_penthouse,
    rd.is_mortgage_allowed,
    rd.is_premium,
    rd.is_emergency,
    
    od.deal_type,
    od.flat_type,
    od.payment_period,
    od.deposit,
    od.prepay_months,
    od.utilities_included,
    od.client_fee,
    od.agent_fee,
    od.description,
    
    d.name as developer_name,
    d.review_count as developer_review_count,
    d.total_rate as developer_rate,
    d.buildings_count as developer_buildings_count,
    d.foundation_year as developer_foundation_year,
    d.is_reliable as developer_is_reliable
FROM offers o
LEFT JOIN addresses a ON o.cian_id = a.cian_id
LEFT JOIN realty_inside ri ON o.cian_id = ri.cian_id
LEFT JOIN realty_outside ro ON o.cian_id = ro.cian_id
LEFT JOIN realty_details rd ON o.cian_id = rd.cian_id
LEFT JOIN offers_details od ON o.cian_id = od.cian_id
LEFT JOIN developers d ON o.cian_id = d.cian_id
"""

print("Загрузка данных из БД...")
df = pd.read_sql(query, engine)
print(f"Загружено строк: {len(df)}")

if 'category' in df.columns:
    print(f"\nРаспределение по category:")
    print(df['category'].value_counts())
    
    initial_count = len(df)
    df = df[df['category'] != 'dailyFlatRent'].copy()
    filtered_count = initial_count - len(df)
    
    if filtered_count > 0:
        print(f"\nОтфильтровано dailyFlatRent: {filtered_count} записей")
        print(f"Осталось после фильтрации: {len(df)} записей")
    else:
        print(f"\ndailyFlatRent не найден в данных")

df.head()


Загрузка данных из БД...


Загружено строк: 22836

Распределение по category:
category
flatRent               21932
dailyFlatRent            856
flatSale                  41
newBuildingFlatSale        7
Name: count, dtype: int64

Отфильтровано dailyFlatRent: 856 записей
Осталось после фильтрации: 21980 записей


,cian_id,price,price_changes,category,views_count,photos_count,floor_number,floors_count,publication_at,offer_created_at,...,utilities_included,client_fee,agent_fee,description,developer_name,developer_review_count,developer_rate,developer_buildings_count,developer_foundation_year,developer_is_reliable
0,325573108,122400.0,"[{""priceData"": {""price"": 122400, ""currency"": ""...",flatRent,None,22.0,3,28,1.767697e+09,2026-01-09 15:35:40,...,1.0,0.0,0.0,Идут показы с датой заселения после 21.01.2026...,Галс-Девелопмент,194.0,4.7,178.0,1994.0,0.0
1,325353038,1200000.0,"[{""priceData"": {""price"": 1200000, ""currency"": ...",flatRent,None,48.0,12,18,1.766627e+09,2026-01-09 15:35:54,...,1.0,0.0,0.0,Лот 554223. Бонус коллегам! Предлагается в аре...,None,NaN,NaN,NaN,NaN,NaN
2,325047081,225000.0,"[{""priceData"": {""price"": 225000, ""currency"": ""...",flatRent,None,11.0,3,5,1.765795e+09,2026-01-09 15:36:41,...,1.0,0.0,0.0,Просторная квартира с 3 спальнями площадью 100...,None,NaN,NaN,NaN,NaN,NaN
3,325322890,1200000.0,"[{""priceData"": {""price"": 1200000, ""currency"": ...",flatRent,None,25.0,12,18,1.766530e+09,2026-01-09 15:46:19,...,1.0,0.0,0.0,Просторная квартира с 4 спальнями общей площад...,None,NaN,NaN,NaN,NaN,NaN
4,325225143,55000.0,"[{""priceData"": {""price"": 55000, ""currency"": ""r...",flatRent,None,20.0,3,5,1.766241e+09,2026-01-09 15:50:22,...,1.0,60.0,50.0,Сдаётся просторная 2-комнатная квартира с евро...,None,NaN,NaN,NaN,NaN,NaN


### 1.1 Анализ price_changes и актуальной цены

Проверяем, актуальная ли цена в `offers.price` или нужно брать из `price_changes`


In [4]:
# Анализ price_changes - извлечение актуальной цены
import json

def get_latest_price_from_changes(price_changes_json):
    """Извлекает последнюю цену из price_changes по дате changeTime"""
    if not price_changes_json:
        return None
    try:
        if isinstance(price_changes_json, str):
            changes = json.loads(price_changes_json)
        else:
            changes = price_changes_json
            
        if not isinstance(changes, list) or len(changes) == 0:
            return None
        
        # Сортируем по changeTime (последняя дата = актуальная цена)
        sorted_changes = sorted(
            changes, 
            key=lambda x: x.get('changeTime', ''),
            reverse=True
        )
        
        latest = sorted_changes[0]
        return latest.get('priceData', {}).get('price')
    except Exception as e:
        return None

# Извлечение актуальной цены из price_changes
if 'price_changes' not in df.columns:
    print("price_changes не найдена, используем offers.price")
    df['price_from_changes'] = None
else:
    df['price_from_changes'] = df['price_changes'].apply(get_latest_price_from_changes)

# Сравнение цен
df['price_diff'] = df['price'] - pd.to_numeric(df['price_from_changes'], errors='coerce')
df['price_matches'] = (abs(df['price_diff']) < 0.01) | (df['price_from_changes'].isna())

if 'price_changes' in df.columns:
    print(f"Записей с price_changes: {df['price_changes'].notna().sum()} ({df['price_changes'].notna().sum()/len(df)*100:.1f}%)")
    print(f"Записей с актуальной ценой из price_changes: {df['price_from_changes'].notna().sum()} ({df['price_from_changes'].notna().sum()/len(df)*100:.1f}%)")
    print(f"\nСравнение цен:")
    print(f"  Цены совпадают: {df['price_matches'].sum()} ({df['price_matches'].sum()/len(df)*100:.1f}%)")
    print(f"  Цены различаются: {(~df['price_matches']).sum()} ({(~df['price_matches']).sum()/len(df)*100:.1f}%)")
    
    if (~df['price_matches']).sum() > 0:
        print(f"\nПримеры различий (первые 5):")
        diff_examples = df[~df['price_matches']][['cian_id', 'price', 'price_from_changes', 'price_diff']].head()
        print(diff_examples.to_string())
    
    df['price_actual'] = df['price_from_changes'].fillna(df['price'])
    print(f"\nАктуальная цена:")
    print(f"  Из price_changes: {(df['price_actual'] == df['price_from_changes']).sum()}")
    print(f"  Из offers.price: {(df['price_actual'] == df['price']).sum()}")
else:
    df['price_actual'] = df['price']


Записей с price_changes: 21829 (99.3%)
Записей с актуальной ценой из price_changes: 21829 (99.3%)

Сравнение цен:
  Цены совпадают: 21924 (99.7%)
  Цены различаются: 56 (0.3%)

Примеры различий (первые 5):
       cian_id        price  price_from_changes   price_diff
633  154723241   15500000.0          14500000.0    1000000.0
638  209040594  288586119.0           3689100.0  284897019.0
639  217791998  482267606.0           6165000.0  476102606.0
641  325646115   45000000.0          38000000.0    7000000.0
652  309118675  406778840.0           5200000.0  401578840.0

Актуальная цена:
  Из price_changes: 21829
  Из offers.price: 21924


## 2. Общий обзор датасета


In [5]:
print(f"Размер датасета: {df.shape}")
print(f"\nКолонки ({len(df.columns)}):")
print(df.columns.tolist())
print(f"\nТипы данных:")
print(df.dtypes)
print(f"\nОсновная информация:")
df.info()


Размер датасета: (21980, 66)

Колонки (66):
['cian_id', 'price', 'price_changes', 'category', 'views_count', 'photos_count', 'floor_number', 'floors_count', 'publication_at', 'offer_created_at', 'offer_updated_at', 'county', 'district', 'street', 'house', 'metro', 'travel_type', 'travel_time', 'coordinates', 'repair_type', 'total_area', 'living_area', 'kitchen_area', 'ceiling_height', 'balconies', 'loggias', 'rooms_count', 'separated_wc', 'combined_wc', 'windows_view', 'build_year', 'entrances', 'material_type', 'parking_type', 'garbage_chute', 'lifts_count', 'passenger_lifts', 'cargo_lifts', 'realty_type', 'project_type', 'heat_type', 'gas_type', 'is_apartment', 'is_penthouse', 'is_mortgage_allowed', 'is_premium', 'is_emergency', 'deal_type', 'flat_type', 'payment_period', 'deposit', 'prepay_months', 'utilities_included', 'client_fee', 'agent_fee', 'description', 'developer_name', 'developer_review_count', 'developer_rate', 'developer_buildings_count', 'developer_foundation_year', 'de

In [6]:
# Статистика по числовым признакам
numeric_cols = df.select_dtypes(include=[np.number]).columns
print("Описательная статистика числовых признаков:")
df[numeric_cols].describe()


Описательная статистика числовых признаков:


,cian_id,price,photos_count,floor_number,floors_count,publication_at,travel_time,total_area,living_area,kitchen_area,...,client_fee,agent_fee,developer_review_count,developer_rate,developer_buildings_count,developer_foundation_year,developer_is_reliable,price_from_changes,price_diff,price_actual
count,2.198000e+04,2.198000e+04,21886.000000,21980.000000,21980.000000,2.197300e+04,21807.000000,21980.000000,16559.000000,18073.000000,...,21932.000000,21932.000000,9148.000000,9148.000000,9285.000000,9151.000000,9362.0,2.182900e+04,2.182900e+04,2.198000e+04
mean,3.206132e+08,3.510220e+05,16.935438,9.029982,17.354595,1.759223e+09,10.688311,58.890823,32.055595,10.750694,...,35.230941,30.905435,2047.798098,4.243747,3549.910932,1999.189706,0.0,2.828490e+05,5.360626e+04,2.977840e+05
std,2.536954e+07,8.056219e+06,8.724388,7.974136,11.668582,4.476987e+07,5.612059,384.565986,26.887362,6.301751,...,29.577853,31.482766,2310.841176,0.667008,4179.198657,10.096597,0.0,6.333057e+06,4.635777e+06,6.558454e+06
min,1.667838e+06,2.300000e+03,1.000000,-2.000000,1.000000,1.255421e+09,1.000000,6.000000,0.200000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1921.000000,0.0,2.300000e+03,-4.360000e+05,2.300000e+03
25%,3.260333e+08,5.700000e+04,10.000000,4.000000,9.000000,1.768999e+09,6.000000,35.000000,19.000000,7.000000,...,0.000000,0.000000,167.000000,4.200000,148.000000,1994.000000,0.0,5.700000e+04,0.000000e+00,5.700000e+04
50%,3.263985e+08,7.500000e+04,15.000000,7.000000,16.000000,1.769941e+09,10.000000,43.000000,24.000000,10.000000,...,50.000000,40.000000,805.000000,4.400000,1371.000000,1994.000000,0.0,7.500000e+04,0.000000e+00,7.500000e+04
75%,3.266247e+08,1.150000e+05,21.000000,12.000000,22.000000,1.770539e+09,14.000000,60.000000,35.000000,12.000000,...,50.000000,50.000000,3904.000000,4.500000,8968.000000,2005.000000,0.0,1.150000e+05,0.000000e+00,1.150000e+05
max,3.269146e+08,8.055300e+08,50.000000,81.000000,117.000000,1.771169e+09,27.000000,56665.000000,500.000000,150.000000,...,100.000000,100.000000,5871.000000,5.000000,19821.000000,2022.000000,0.0,8.055300e+08,4.761026e+08,8.055300e+08


In [7]:
# Используем актуальную цену для анализа (из price_changes или offers.price)
if 'price_actual' not in df.columns:
    # Если ячейка 6 не выполнилась, используем offers.price
    df['price'] = pd.to_numeric(df['price'], errors='coerce')
else:
    df['price'] = pd.to_numeric(df['price_actual'], errors='coerce')

fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Гистограмма
axes[0, 0].hist(df['price'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0, 0].set_title('Распределение цен (гистограмма)')
axes[0, 0].set_xlabel('Цена (руб.)')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].grid(True, alpha=0.3)

# Boxplot
axes[0, 1].boxplot(df['price'].dropna(), vert=True)
axes[0, 1].set_title('Распределение цен (boxplot)')
axes[0, 1].set_ylabel('Цена (руб.)')
axes[0, 1].grid(True, alpha=0.3)

# Логарифмированная шкала
log_price = np.log1p(df['price'].dropna())
axes[1, 0].hist(log_price, bins=50, edgecolor='black', alpha=0.7)
axes[1, 0].set_title('Распределение цен (логарифмированная шкала)')
axes[1, 0].set_xlabel('log(Цена + 1)')
axes[1, 0].set_ylabel('Частота')
axes[1, 0].grid(True, alpha=0.3)

# Q-Q plot для проверки нормальности
from scipy import stats
stats.probplot(df['price'].dropna(), dist="norm", plot=axes[1, 1])
axes[1, 1].set_title('Q-Q plot цены')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Статистика по цене:")
print(df['price'].describe())
print(f"\nМедиана: {df['price'].median():.2f}")
print(f"Среднее: {df['price'].mean():.2f}")
print(f"Мода: {df['price'].mode().values[0] if len(df['price'].mode()) > 0 else 'N/A'}")


Статистика по цене:
count    2.198000e+04
mean     2.977840e+05
std      6.558454e+06
min      2.300000e+03
25%      5.700000e+04
50%      7.500000e+04
75%      1.150000e+05
max      8.055300e+08
Name: price, dtype: float64

Медиана: 75000.00
Среднее: 297783.97
Мода: 60000.0


### 3.2 Распределение числовых признаков


In [8]:
key_numeric = ['total_area', 'living_area', 'kitchen_area', 'floor_number', 
               'floors_count', 'build_year', 'rooms_count', 'ceiling_height']
key_numeric = [col for col in key_numeric if col in df.columns]

fig, axes = plt.subplots(len(key_numeric), 2, figsize=(15, 4 * len(key_numeric)))

for i, col in enumerate(key_numeric):
    if col in df.columns:
        data = pd.to_numeric(df[col], errors='coerce').dropna()
        
        if len(data) > 0:
            # Гистограмма
            axes[i, 0].hist(data, bins=30, edgecolor='black', alpha=0.7)
            axes[i, 0].set_title(f'Распределение {col}')
            axes[i, 0].set_xlabel(col)
            axes[i, 0].set_ylabel('Частота')
            axes[i, 0].grid(True, alpha=0.3)
            
            # Boxplot
            axes[i, 1].boxplot(data, vert=True)
            axes[i, 1].set_title(f'Boxplot {col}')
            axes[i, 1].set_ylabel(col)
            axes[i, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### 3.3 Распределение категориальных признаков


In [9]:
cat_cols = ['category', 'district', 'repair_type', 'material_type', 'realty_type', 
            'deal_type', 'flat_type', 'rooms_count']
cat_cols = [col for col in cat_cols if col in df.columns]

fig, axes = plt.subplots(len(cat_cols), 1, figsize=(15, 5 * len(cat_cols)))

for i, col in enumerate(cat_cols):
    if col in df.columns:
        value_counts = df[col].value_counts().head(15)
        
        axes[i].barh(range(len(value_counts)), value_counts.values)
        axes[i].set_yticks(range(len(value_counts)))
        axes[i].set_yticklabels(value_counts.index)
        axes[i].set_title(f'{col} (топ-15)')
        axes[i].set_xlabel('Количество')
        axes[i].grid(True, alpha=0.3, axis='x')
        
        axes[i].invert_yaxis()

plt.tight_layout()
plt.show()

# Выводим статистику по категориальным признакам
for col in cat_cols:
    if col in df.columns:
        print(f"\n{col}:")
        print(f"  Уникальных значений: {df[col].nunique()}")
        print(f"  Пропусков: {df[col].isna().sum()} ({df[col].isna().sum() / len(df) * 100:.1f}%)")
        print(f"  Топ-5:")
        print(df[col].value_counts().head())



category:
  Уникальных значений: 3
  Пропусков: 0 (0.0%)
  Топ-5:
category
flatRent               21932
flatSale                  41
newBuildingFlatSale        7
Name: count, dtype: int64

district:
  Уникальных значений: 126
  Пропусков: 2243 (10.2%)
  Топ-5:
district
Пресненский            840
Хорошево-Мневники      598
Раменки                519
Хорошевский            452
Очаково-Матвеевское    450
Name: count, dtype: int64

repair_type:
  Уникальных значений: 4
  Пропусков: 998 (4.5%)
  Топ-5:
repair_type
euro        9078
design      6338
cosmetic    5293
no           273
Name: count, dtype: int64

material_type:
  Уникальных значений: 8
  Пропусков: 2065 (9.4%)
  Топ-5:
material_type
none        5460
panel       5244
monolith    4412
brick       3201
block       1201
Name: count, dtype: int64

realty_type:
  Уникальных значений: 1
  Пропусков: 0 (0.0%)
  Топ-5:
realty_type
flat    21980
Name: count, dtype: int64

deal_type:
  Уникальных значений: 2
  Пропусков: 0 (0.0%)
  Топ-5:


## 4. Анализ пропусков


In [10]:
missing = df.isnull().sum()
missing_percent = (missing / len(df)) * 100
missing_df = pd.DataFrame({
    'Пропусков': missing,
    'Процент': missing_percent
}).sort_values('Процент', ascending=False)

print("Колонки с пропусками:")
print(missing_df[missing_df['Пропусков'] > 0])

fig, axes = plt.subplots(1, 2, figsize=(20, 8))
cols_with_missing = missing_df[missing_df['Пропусков'] > 0].index[:30]
if len(cols_with_missing) > 0:
    sns.heatmap(df[cols_with_missing].isnull(), yticklabels=False, 
                cbar=True, cmap='viridis', ax=axes[0])
    axes[0].set_title('Heatmap пропусков (топ-30 колонок)')
    
    top_missing = missing_df.head(20)
    axes[1].barh(range(len(top_missing)), top_missing['Процент'].values)
    axes[1].set_yticks(range(len(top_missing)))
    axes[1].set_yticklabels(top_missing.index, fontsize=8)
    axes[1].set_xlabel('Процент пропусков')
    axes[1].set_title('Топ-20 колонок по проценту пропусков')
    axes[1].grid(True, alpha=0.3, axis='x')
    axes[1].invert_yaxis()

plt.tight_layout()
plt.show()


Колонки с пропусками:
                           Пропусков     Процент
views_count                    21980  100.000000
is_mortgage_allowed            21936   99.799818
is_penthouse                   19865   90.377616
developer_rate                 12832   58.380346
developer_review_count         12832   58.380346
developer_foundation_year      12829   58.366697
developer_buildings_count      12695   57.757052
developer_name                 12695   57.757052
developer_is_reliable          12618   57.406733
garbage_chute                  11133   50.650591
project_type                    8670   39.444950
windows_view                    7760   35.304823
ceiling_height                  7343   33.407643
cargo_lifts                     6017   27.374886
entrances                       5944   27.042766
living_area                     5421   24.663330
parking_type                    4976   22.638763
kitchen_area                    3907   17.775250
lifts_count                     3562   16.20564

### Как заполнять будем пропуски

- **Числовые признаки**: медиана или среднее, в зависимости от распределения
- **Категориальные признаки**: мода или "unknown"
- **Координаты**: могут быть критичными для geo-фичей
- **build_year**: можно попробовать восстановить по району/материалу


## 5. Выбросы

### 5.1 IQR метод


In [11]:
def detect_outliers_iqr(df, column):
    """Обнаружение выбросов методом IQR"""
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[column] < lower_bound) | (df[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

key_columns = ['price', 'total_area', 'living_area', 'floor_number', 'build_year']
outliers_summary = {}
for col in key_columns:
    if col in df.columns:
        df_col = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(df_col) > 0:
            outliers, lower, upper = detect_outliers_iqr(df, col)
            outliers_summary[col] = {
                'count': len(outliers),
                'percent': len(outliers) / len(df) * 100,
                'lower_bound': lower,
                'upper_bound': upper,
                'min_value': df[col].min(),
                'max_value': df[col].max()
            }

print("Выбросы (IQR метод):")
for col, stats in outliers_summary.items():
    print(f"\n{col}:")
    print(f"  Выбросов: {stats['count']} ({stats['percent']:.1f}%)")
    print(f"  Границы: [{stats['lower_bound']:.2f}, {stats['upper_bound']:.2f}]")
    print(f"  Диапазон данных: [{stats['min_value']:.2f}, {stats['max_value']:.2f}]")

fig, axes = plt.subplots(len(outliers_summary), 1, figsize=(12, 4 * len(outliers_summary)))

for i, (col, stats) in enumerate(outliers_summary.items()):
    data = pd.to_numeric(df[col], errors='coerce').dropna()
    axes[i].boxplot(data, vert=True)
    axes[i].axhline(y=stats['lower_bound'], color='r', linestyle='--', alpha=0.5, label='Lower bound')
    axes[i].axhline(y=stats['upper_bound'], color='r', linestyle='--', alpha=0.5, label='Upper bound')
    axes[i].set_title(f'Выбросы в {col} (IQR метод)')
    axes[i].set_ylabel(col)
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


Выбросы (IQR метод):

price:
  Выбросов: 2469 (11.2%)
  Границы: [-30000.00, 202000.00]
  Диапазон данных: [2300.00, 805530000.00]

total_area:
  Выбросов: 2003 (9.1%)
  Границы: [-2.50, 97.50]
  Диапазон данных: [6.00, 56665.00]

living_area:
  Выбросов: 1443 (6.6%)
  Границы: [-5.00, 59.00]
  Диапазон данных: [0.20, 500.00]

floor_number:
  Выбросов: 985 (4.5%)
  Границы: [-8.00, 24.00]
  Диапазон данных: [-2.00, 81.00]

build_year:
  Выбросов: 36 (0.2%)
  Границы: [1896.00, 2096.00]
  Диапазон данных: [1785.00, 2027.00]


### 5.2 Z-score метод


In [12]:
def detect_outliers_zscore(df, column, threshold=3):
    """Обнаружение выбросов методом Z-score"""
    df_col = pd.to_numeric(df[column], errors='coerce')
    z_scores = np.abs((df_col - df_col.mean()) / df_col.std())
    outliers = df[z_scores > threshold]
    return outliers, z_scores

zscore_summary = {}
for col in key_columns:
    if col in df.columns:
        df_col = pd.to_numeric(df[col], errors='coerce').dropna()
        if len(df_col) > 0 and df_col.std() > 0:
            outliers, z_scores = detect_outliers_zscore(df, col, threshold=3)
            zscore_summary[col] = {
                'count': len(outliers),
                'percent': len(outliers) / len(df) * 100,
                'max_z_score': z_scores.max() if len(z_scores) > 0 else 0
            }

print("Выбросы (Z-score метод, threshold=3):")
for col, stats in zscore_summary.items():
    print(f"{col}: {stats['count']} выбросов ({stats['percent']:.1f}%), max Z-score: {stats['max_z_score']:.2f}")


Выбросы (Z-score метод, threshold=3):
price: 41 выбросов (0.2%), max Z-score: 122.78
total_area: 1 выбросов (0.0%), max Z-score: 147.19
living_area: 344 выбросов (1.6%), max Z-score: 17.40
floor_number: 369 выбросов (1.7%), max Z-score: 9.03
build_year: 158 выбросов (0.7%), max Z-score: 7.38


### 5.3 Анализ экстремальных значений цены

Проверим самые дорогие и дешевые предложения


In [13]:
print("Топ-10 самых дорогих:")
print(df.nlargest(10, 'price')[['cian_id', 'price', 'total_area', 'district', 'rooms_count', 'repair_type']].to_string())

print("\nТоп-10 самых дешевых:")
print(df.nsmallest(10, 'price')[['cian_id', 'price', 'total_area', 'district', 'rooms_count', 'repair_type']].to_string())

df['price_per_sqm'] = df['price'] / pd.to_numeric(df['total_area'], errors='coerce')

print("\nСтатистика цены за м²:")
print(df['price_per_sqm'].describe())

fig, axes = plt.subplots(1, 2, figsize=(15, 5))
axes[0].hist(df['price_per_sqm'].dropna(), bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение цены за м²')
axes[0].set_xlabel('Цена за м² (руб.)')
axes[0].set_ylabel('Частота')
axes[0].grid(True, alpha=0.3)

axes[1].boxplot(df['price_per_sqm'].dropna(), vert=True)
axes[1].set_title('Boxplot цены за м²')
axes[1].set_ylabel('Цена за м² (руб.)')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


Топ-10 самых дорогих:
         cian_id        price  total_area      district  rooms_count repair_type
648    313037687  805530000.0       244.1   Пресненский          2.0        None
630      4349182  236557541.0       224.6   Гагаринский          4.0          no
635    179176213  170000000.0       164.0     Хамовники          5.0        euro
653    324864015  170000000.0       120.0      Якиманка          3.0      design
634    169579402  155000000.0       170.0         Арбат          3.0      design
646    286258299  148230309.0       104.2      Тверской          2.0        None
643    322430288  128304720.0        90.4  Дорогомилово          2.0        None
629      3184068  119000000.0       220.0     Мещанский          5.0      design
654    307211222  115783668.0        57.6      Тверской          1.0        None
18293  326667118   99000000.0        41.0   Даниловский          1.0        euro

Топ-10 самых дешевых:
        cian_id    price  total_area              district  room

## 6. Утечка по времени

Проверяем, что train и test разделены по времени публикации (test позже train).


In [14]:
df['publication_date'] = pd.to_datetime(df['publication_at'], unit='s', errors='coerce')
df_with_date = df.dropna(subset=['publication_date']).copy()

print(f"Всего записей: {len(df)}")
print(f"С датой публикации: {len(df_with_date)}")
print(f"\nДиапазон дат:")
print(f"  От: {df_with_date['publication_date'].min()}")
print(f"  До: {df_with_date['publication_date'].max()}")

fig, axes = plt.subplots(2, 1, figsize=(15, 10))
axes[0].hist(df_with_date['publication_date'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_title('Распределение публикаций по времени')
axes[0].set_xlabel('Дата публикации')
axes[0].set_ylabel('Количество')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

split_date = df_with_date['publication_date'].max() - pd.Timedelta(days=30)
train_mask = df_with_date['publication_date'] <= split_date
test_mask = df_with_date['publication_date'] > split_date

train_data = df_with_date[train_mask]
test_data = df_with_date[test_mask]

print(f"\nРазделение на train/test (split_date = {split_date.date()}):")
print(f"  Train: {len(train_data)} записей ({len(train_data)/len(df_with_date)*100:.1f}%)")
print(f"  Test: {len(test_data)} записей ({len(test_data)/len(df_with_date)*100:.1f}%)")

axes[1].scatter(train_data['publication_date'], train_data['price'], 
                alpha=0.3, label='Train', s=10)
axes[1].scatter(test_data['publication_date'], test_data['price'], 
                alpha=0.3, label='Test', s=10, color='red')
axes[1].axvline(x=split_date, color='green', linestyle='--', linewidth=2, label='Split date')
axes[1].set_title('Разделение train/test по времени')
axes[1].set_xlabel('Дата публикации')
axes[1].set_ylabel('Цена (руб.)')
axes[1].legend()
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

if len(train_data) > 0 and len(test_data) > 0:
    max_train_date = train_data['publication_date'].max()
    min_test_date = test_data['publication_date'].min()
    print(f"\nПроверка временного разделения:")
    print(f"  Max train date: {max_train_date}")
    print(f"  Min test date: {min_test_date}")
    print(f"  Test позже train: {min_test_date > max_train_date}")


Всего записей: 21980
С датой публикации: 21973

Диапазон дат:
  От: 2009-10-13 07:59:21
  До: 2026-02-15 15:30:27

Разделение на train/test (split_date = 2026-01-16):
  Train: 5296 записей (24.1%)
  Test: 16677 записей (75.9%)



Проверка временного разделения:
  Max train date: 2026-01-16 14:55:01
  Min test date: 2026-01-16 15:30:53
  Test позже train: True


## 7. Корреляции

Анализ взаимосвязей между признаками


In [15]:
numeric_for_corr = ['price', 'total_area', 'living_area', 'kitchen_area', 
                    'floor_number', 'floors_count', 'build_year', 'rooms_count',
                    'ceiling_height', 'views_count', 'photos_count']
numeric_for_corr = [col for col in numeric_for_corr if col in df.columns]

corr_matrix = df[numeric_for_corr].apply(pd.to_numeric, errors='coerce').corr()
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Корреляционная матрица числовых признаков')
plt.tight_layout()
plt.show()

if 'price' in corr_matrix.columns:
    price_corr = corr_matrix['price'].sort_values(ascending=False)
    print("\nКорреляции с ценой:")
    print(price_corr)



Корреляции с ценой:
price             1.000000
living_area       0.086028
kitchen_area      0.067708
rooms_count       0.039658
photos_count      0.018864
total_area        0.008322
build_year        0.003670
ceiling_height    0.001090
floor_number     -0.005563
floors_count     -0.009552
views_count            NaN
Name: price, dtype: float64


## 9. Краткий анализ результатов EDA

### Основные находки по каждой ячейке:


In [16]:
# Итоговый анализ результатов
print("=" * 50)
print("АНАЛИЗ РЕЗУЛЬТАТОВ EDA")
print("=" * 50)

print(f"\n1. Размер датасета:")
print(f"   Записей: {len(df)}, колонок: {len(df.columns)}")

if 'category' in df.columns:
    print(f"\n   Фильтрация category:")
    category_counts = df['category'].value_counts()
    print(f"   {category_counts.to_dict()}")
    if 'dailyFlatRent' in category_counts.index:
        print(f"   dailyFlatRent: {category_counts['dailyFlatRent']} записей (отфильтровано)")

print(f"\n2. Цены:")
if 'price_actual' in df.columns:
    price_col = 'price_actual'
else:
    price_col = 'price'
    
price_stats = df[price_col].describe()
print(f"   Медиана: {price_stats['50%']:,.0f} руб")
print(f"   Среднее: {price_stats['mean']:,.0f} руб")
print(f"   Min: {price_stats['min']:,.0f} руб")
print(f"   Max: {price_stats['max']:,.0f} руб")
print(f"   Max цена {price_stats['max']:,.0f} руб - выброс")
print(f"   Std: {price_stats['std']:,.0f} руб")

if 'price_changes' in df.columns:
    price_changes_count = df['price_changes'].notna().sum()
    price_matches_count = df['price_matches'].sum() if 'price_matches' in df.columns else 0
    print(f"\n   Анализ price_changes:")
    print(f"   Записей с price_changes: {price_changes_count} ({price_changes_count/len(df)*100:.1f}%)")
    print(f"   Цены совпадают: {price_matches_count} ({price_matches_count/len(df)*100:.1f}%)")
    if 'price_actual' in df.columns:
        from_changes = (df['price_actual'] == df['price_from_changes']).sum() if 'price_from_changes' in df.columns else 0
        from_offers = (df['price_actual'] == df['price']).sum() if 'price' in df.columns else 0
        print(f"   Используется: price_changes={from_changes}, offers.price={from_offers}")

if 'price_per_sqm' in df.columns:
    price_sqm = df['price_per_sqm'].dropna()
    if len(price_sqm) > 0:
        print(f"\n   Цена за м²:")
        print(f"   Медиана: {price_sqm.median():,.0f} руб/м²")
        print(f"   Mean: {price_sqm.mean():,.0f} руб/м²")
        print(f"   Max: {price_sqm.max():,.0f} руб/м² (выброс)")

# Проверка выбросов
Q1 = price_stats['25%']
Q3 = price_stats['75%']
IQR = Q3 - Q1
outliers_count = len(df[(df[price_col] < Q1 - 1.5*IQR) | (df[price_col] > Q3 + 1.5*IQR)])
print(f"   Выбросов (IQR): {outliers_count} ({outliers_count/len(df)*100:.1f}%)")

print(f"\n3. Пропуски (топ-10):")
missing_top10 = df.isnull().sum().sort_values(ascending=False).head(10)
for col, count in missing_top10.items():
    if col not in ['price_diff', 'price_from_changes', 'price_matches', 'price_changes']:
        print(f"   {col}: {count} ({count/len(df)*100:.1f}%)")

print(f"\n4. Корреляции с ценой:")
if 'total_area' in df.columns and price_col in df.columns:
    corr_total = df[['total_area', price_col]].apply(pd.to_numeric, errors='coerce').corr().iloc[0, 1]
    print(f"   total_area: {corr_total:.4f}")
if 'kitchen_area' in df.columns and price_col in df.columns:
    corr_kitchen = df[['kitchen_area', price_col]].apply(pd.to_numeric, errors='coerce').corr().iloc[0, 1]
    print(f"   kitchen_area: {corr_kitchen:.4f}")

print(f"\n5. Категориальные признаки:")
if 'category' in df.columns:
    print(f"   category:")
    print(f"      {df['category'].value_counts().to_dict()}")

print(f"   deal_type:")
if 'deal_type' in df.columns:
    print(f"      {df['deal_type'].value_counts().to_dict()}")
    if 'sale' in df['deal_type'].values:
        sale_count = len(df[df['deal_type'] == 'sale'])
        if sale_count > 0:
            print(f"      sale: {sale_count} записей (нужно фильтровать)")

print(f"\n6. Временное разделение:")
if 'publication_date' in df.columns:
    df_with_date = df.dropna(subset=['publication_date'])
    print(f"   Диапазон: {df_with_date['publication_date'].min()} - {df_with_date['publication_date'].max()}")
    split_date = df_with_date['publication_date'].max() - pd.Timedelta(days=30)
    train_size = len(df_with_date[df_with_date['publication_date'] <= split_date])
    test_size = len(df_with_date[df_with_date['publication_date'] > split_date])
    print(f"   Train/Test (30 дней): {train_size}/{test_size} ({train_size/len(df_with_date)*100:.1f}%/{test_size/len(df_with_date)*100:.1f}%)")

print(f"\n7. Критические проблемы:")
issues = []
if price_stats['max'] > 100000000:
    issues.append(f"Выбросы в цене: max={price_stats['max']:,.0f} руб")
if 'deal_type' in df.columns and (df['deal_type'] == 'sale').any():
    issues.append(f"deal_type='sale': {len(df[df['deal_type'] == 'sale'])} шт")
if 'total_area' in df.columns and df['total_area'].isna().sum() / len(df) > 0.1:
    issues.append(f"Пропуски в total_area: {df['total_area'].isna().sum()/len(df)*100:.1f}%")
if 'price_changes' in df.columns and 'price_matches' in df.columns:
    diff_count = (~df['price_matches']).sum()
    if diff_count > 0:
        issues.append(f"Различия price_changes и offers.price: {diff_count} записей (0.2%)")
if 'category' in df.columns and (df['category'] == 'dailyFlatRent').any():
    daily_count = len(df[df['category'] == 'dailyFlatRent'])
    issues.append(f"dailyFlatRent в данных: {daily_count} записей (нужно фильтровать)")

if issues:
    for i, issue in enumerate(issues, 1):
        print(f"   {i}. {issue}")
else:
    print("   Не найдено критических проблем")

print("\n" + "=" * 50)


АНАЛИЗ РЕЗУЛЬТАТОВ EDA

1. Размер датасета:
   Записей: 21980, колонок: 68

   Фильтрация category:
   {'flatRent': 21932, 'flatSale': 41, 'newBuildingFlatSale': 7}

2. Цены:
   Медиана: 75,000 руб
   Среднее: 297,784 руб
   Min: 2,300 руб
   Max: 805,530,000 руб
   Max цена 805,530,000 руб - выброс
   Std: 6,558,454 руб

   Анализ price_changes:
   Записей с price_changes: 21829 (99.3%)
   Цены совпадают: 21924 (99.7%)
   Используется: price_changes=21829, offers.price=21980

   Цена за м²:
   Медиана: 1,774 руб/м²
   Mean: 3,906 руб/м²
   Max: 3,300,000 руб/м² (выброс)
   Выбросов (IQR): 2469 (11.2%)

3. Пропуски (топ-10):
   views_count: 21980 (100.0%)
   is_mortgage_allowed: 21936 (99.8%)
   is_penthouse: 19865 (90.4%)
   developer_review_count: 12832 (58.4%)
   developer_rate: 12832 (58.4%)
   developer_foundation_year: 12829 (58.4%)
   developer_buildings_count: 12695 (57.8%)
   developer_name: 12695 (57.8%)
   developer_is_reliable: 12618 (57.4%)
   garbage_chute: 11133 (50.7%)


   Train/Test (30 дней): 5296/16677 (24.1%/75.9%)

7. Критические проблемы:
   1. Выбросы в цене: max=805,530,000 руб
   2. deal_type='sale': 48 шт
   3. Различия price_changes и offers.price: 56 записей (0.2%)



## 8. Анализ дубликатов

Проверка дубликатов объявлений по этаж + адрес + площадь + комнаты


In [17]:
df['floor_number'] = pd.to_numeric(df['floor_number'], errors='coerce')
df['total_area'] = pd.to_numeric(df['total_area'], errors='coerce')
df['rooms_count'] = pd.to_numeric(df['rooms_count'], errors='coerce')

df['street'] = df['street'].fillna('').astype(str).str.strip().str.lower()
df['house'] = df['house'].fillna('').astype(str).str.strip().str.lower()

df['address_key'] = df['street'].fillna('') + '_' + df['house'].fillna('')
df['area_rounded'] = df['total_area'].round(0)
df['rooms_rounded'] = df['rooms_count'].round(0)

df['duplicate_key'] = (
    df['address_key'].astype(str) + '_' +
    df['floor_number'].astype(str) + '_' +
    df['area_rounded'].astype(str) + '_' +
    df['rooms_rounded'].astype(str)
)

duplicates = df[df.duplicated(subset=['duplicate_key'], keep=False)].sort_values('duplicate_key')

print(f"Всего записей: {len(df)}")
print(f"Уникальных ключей: {df['duplicate_key'].nunique()}")
print(f"Дубликатов: {len(df) - df['duplicate_key'].nunique()}")

if len(duplicates) > 0:
    print(f"\nПримеры дубликатов (первые 10):")
    print(duplicates[['cian_id', 'address_key', 'floor_number', 'total_area', 'rooms_count', 'price_actual', 'publication_at']].head(10).to_string())
    
    df['publication_date'] = pd.to_datetime(df['publication_at'], unit='s', errors='coerce')
    duplicates_with_date = duplicates.dropna(subset=['publication_date'])
    if len(duplicates_with_date) > 0:
        print(f"\nСтатистика дубликатов:")
        print(f"  Групп дубликатов: {duplicates_with_date.groupby('duplicate_key').size().shape[0]}")
        print(f"  Средний размер группы: {duplicates_with_date.groupby('duplicate_key').size().mean():.1f}")
        print(f"  Максимальный размер группы: {duplicates_with_date.groupby('duplicate_key').size().max()}")
else:
    print("Дубликатов не найдено")


Всего записей: 21980
Уникальных ключей: 20215
Дубликатов: 1765

Примеры дубликатов (первые 10):
         cian_id                      address_key  floor_number  total_area  rooms_count  price_actual  publication_at
9123   326296445     1-й балтийский переулок_3/25             6        80.0          3.0      140000.0    1.769621e+09
8806   326289341     1-й балтийский переулок_3/25             6        80.0          3.0      140000.0    1.769611e+09
17201  326626199        1-й войковский проезд_4к1             3        45.0          2.0       70000.0    1.770546e+09
12414  326424704        1-й войковский проезд_4к1             3        45.0          2.0       65000.0    1.770021e+09
22159  326895616  1-й грайвороновский проезд_13к2            22        45.9          2.0       82000.0    1.771057e+09
1096   325652319  1-й грайвороновский проезд_13к2            22        45.9          2.0       90000.0    1.768178e+09
21571  326846594     1-й грайвороновский проезд_3            22        

## 9. Выводы и рекомендации


In [18]:
print("=" * 60)
print("ВЫВОДЫ EDA")
print("=" * 60)

print(f"\n1. Размер датасета: {len(df)} записей, {len(df.columns)} колонок")

if 'price_actual' in df.columns:
    price_col = 'price_actual'
else:
    price_col = 'price'

price_stats = df[price_col].describe()
print(f"\n2. Цены:")
print(f"   Медиана: {price_stats['50%']:,.0f} руб")
print(f"   Среднее: {price_stats['mean']:,.0f} руб")
print(f"   Min/Max: {price_stats['min']:,.0f} / {price_stats['max']:,.0f} руб")

Q1 = price_stats['25%']
Q3 = price_stats['75%']
IQR = Q3 - Q1
outliers_count = len(df[(df[price_col] < Q1 - 1.5*IQR) | (df[price_col] > Q3 + 1.5*IQR)])
print(f"   Выбросов (IQR): {outliers_count} ({outliers_count/len(df)*100:.1f}%)")

if 'price_changes' in df.columns:
    price_changes_count = df['price_changes'].notna().sum()
    print(f"\n3. price_changes: {price_changes_count} записей ({price_changes_count/len(df)*100:.1f}%)")

if 'duplicate_key' in df.columns:
    duplicates_count = len(df) - df['duplicate_key'].nunique()
    print(f"\n4. Дубликаты: {duplicates_count} записей ({duplicates_count/len(df)*100:.1f}%)")

print(f"\n5. Пропуски (топ-5):")
missing_top5 = df.isnull().sum().sort_values(ascending=False).head(5)
for col, count in missing_top5.items():
    if count > 0:
        print(f"   {col}: {count} ({count/len(df)*100:.1f}%)")

if 'total_area' in df.columns and price_col in df.columns:
    corr_total = df[['total_area', price_col]].apply(pd.to_numeric, errors='coerce').corr().iloc[0, 1]
    print(f"\n6. Корреляции с ценой:")
    print(f"   total_area: {corr_total:.4f}")
    if 'kitchen_area' in df.columns:
        corr_kitchen = df[['kitchen_area', price_col]].apply(pd.to_numeric, errors='coerce').corr().iloc[0, 1]
        print(f"   kitchen_area: {corr_kitchen:.4f}")

print(f"\n7. Рекомендации:")
print("   - Фильтровать category='dailyFlatRent' и deal_type='sale'")
print("   - Удалить выбросы: цена > 10 млн или < 1000 руб, площадь > 500 м² или < 10 м²")
print("   - Удалить дубликаты по этаж + адрес + площадь + комнаты (оставить более позднее)")
print("   - Использовать price_actual из price_changes")
print("   - Корреляции слабые - нужны дополнительные фичи (geo, цена за м²)")
print("=" * 60)


ВЫВОДЫ EDA

1. Размер датасета: 21980 записей, 72 колонок

2. Цены:
   Медиана: 75,000 руб
   Среднее: 297,784 руб
   Min/Max: 2,300 / 805,530,000 руб
   Выбросов (IQR): 2469 (11.2%)

3. price_changes: 21829 записей (99.3%)

4. Дубликаты: 1765 записей (8.0%)

5. Пропуски (топ-5):
   views_count: 21980 (100.0%)
   is_mortgage_allowed: 21936 (99.8%)
   is_penthouse: 19865 (90.4%)
   developer_review_count: 12832 (58.4%)
   developer_rate: 12832 (58.4%)

6. Корреляции с ценой:
   total_area: 0.0083
   kitchen_area: 0.0677

7. Рекомендации:
   - Фильтровать category='dailyFlatRent' и deal_type='sale'
   - Удалить выбросы: цена > 10 млн или < 1000 руб, площадь > 500 м² или < 10 м²
   - Удалить дубликаты по этаж + адрес + площадь + комнаты (оставить более позднее)
   - Использовать price_actual из price_changes
   - Корреляции слабые - нужны дополнительные фичи (geo, цена за м²)
